In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



In [6]:
query_text = f"""
SHOW TABLES
"""
tables = ch.query_run(query_text)['name'].to_list()


In [8]:
filtered_tables = [
    t for t in tables
    if ('_st_' in t or t.startswith('t_')) and not (t.endswith('_ch') or t.endswith('_mv'))
]


filtered_tables_mv_df = [
    t for t in tables
    if t.endswith('_mv_df')
]

filtered_tables

['billing_orders_devices_st_partner',
 'buildings_st_partner',
 'cameras_daily_percentage_online_st_asgard',
 'cameras_st_asgard',
 'cameras_st_partner',
 'categories_st_support',
 'citizen_payments_st_mobile',
 'citizens_st_mobile',
 'companies_st_partner',
 'complaints_st_partner',
 'entries_st_mobile',
 'flats_st_partner',
 'flussonic_stats_st_asgard',
 'gates_st_partner',
 'installation_point_st_partner',
 'intercoms_daily_percentage_online_st_asgard',
 'intercoms_st_asgard',
 'intercoms_st_partner',
 'mv_companies_st_partner',
 'no_video_on_stream_mobile_st_asgard',
 'opendoor_types_mobile_st_asgard',
 'reconnects_intercoms_st_asgard',
 'requests_st_support',
 'sessions_st_mobile',
 'subscriptions_st_mobile',
 'subscriptions_st_mobile_ch_2',
 'subscriptions_st_mobile_mv_2',
 't_activated_citizen_id',
 't_all_installetion_points',
 't_billings_cohorts',
 't_bitrix_deals_sales',
 't_cameras_by_companies',
 't_citizen_id_in_flat_with_subscriptions',
 't_complaints_st_partner',
 't_co

In [ ]:
cols = schema_dr[table]
        group_cols = []
        for col, ctype in cols:
            if col == 'report_date':
                group_cols.append('report_date')
            elif 'Int' in ctype or 'UInt' in ctype:
                group_cols.append(f"sum(if({col} != 0, 1, 0)) as {col}")
            else:
                group_cols.append(f"sum(if({col} != '', 1, 0)) as {col}")
        select_clause = ', '.join(group_cols)

In [ ]:
schema_dr = ch.get_schema_dr('<your_database_name>')

for table in filtered_tables:
    mv_table = f"{table}_mv_dr"
    if mv_exists in filtered_tables_mv_df:
        query_text = f"""
        SYSTEM REFRESH MATERIALIZED VIEW {mv_table}
        """
        ch.query_run(query_text)
    else:
        cols = schema_dr[table]
        group_cols = []
        for col, ctype in cols:
            if col == 'report_date':
                group_cols.append('report_date')
            elif 'Int' in ctype or 'UInt' in ctype:
                group_cols.append(f"sum(if({col} != 0, 1, 0)) as {col}")
            else:
                group_cols.append(f"sum(if({col} != '', 1, 0)) as {col}")
        select_clause = ', '.join(group_cols)
        query_text = f"""
        CREATE MATERIALIZED VIEW {mv_table} 
        ENGINE = AggregatingMergeTree()
        ORDER BY report_date AS
        SELECT {select_clause}
        FROM {table}
        GROUP BY report_date
        """
        ch.query_run(query_text)
